In [1]:

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

training_set = 'dataset/dogs_vs_cats/train'
test_set = 'dataset/dogs_vs_cats/test'



In [43]:
import tensorflow
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow
import ssl


IMAGE_SIZE = 64   
vgg = VGG16(input_shape=[64,64,3], weights='imagenet', include_top=False)
# weights are imported directly from the imagenet classifier
# include_top means whether or not we are going to keep the input and output layers

In [44]:
# this specifies that we will not re-train any layers in the expert model,
# and just use the weights instead
for layer in vgg.layers:
    layer.trainable = False

In [45]:
# this is printing how many folder there are in the training_set
# glob returns how many folders there are in a given directory 
from glob import glob
folders = glob(training_set)
print(len(folders))

1


In [46]:
from keras.models import Model
x = Flatten()(vgg.output) # condenses what ever output we get from the expert model into one dimension
prediction = Dense(2, activation='softmax')(x) #sets up the output layer (in this case only 2 outputs)
model = Model(inputs=vgg.input, outputs=prediction) #specifies input coming from expert model, out creates output prediction
model.summary()



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0   

In [47]:
#compiling model
from keras import optimizers 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [48]:
from keras.applications.vgg16 import preprocess_input
# preprocess_input is the same type of image preproccessing the expert model used

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rotation_range=40, 
                                    width_shift_range=0.2, height_shift_range=0.2, 
                                    shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest') 

# Generating images for the Test set
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rotation_range=40, 
                                    width_shift_range=0.2, height_shift_range=0.2, 
                                    shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

# Creating the Training set
proccessed_training_set = train_datagen.flow_from_directory(training_set,
                                                 target_size = (64, 64),
                                                 class_mode = 'categorical',
                                                 batch_size = 32)
                                                 

                                            
# Creating the Test set
proccessed_test_set = test_datagen.flow_from_directory(test_set,
                                            target_size = (64, 64),
                                            class_mode = 'categorical',
                                            batch_size = 32)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [57]:
# from datetime import datetime
# from keras.callbacks import ModelCheckpoint
# checkpoint = ModelCheckpoint(filepath='mymodel.h5',
#                             verbose=2, save_best_only=True)
# callbacks = [checkpoint]

# start = datetime.now()

model_history = model.fit_generator(
    proccessed_training_set,
    validation_data = proccessed_test_set,
    epochs = 25,
    # steps_per_epoch = 60,
    # validation_steps = 60
    # ,
    # callbacks=callbacks, 
    # verbose = 2, 
)

model.save('VGG16_Cat-Dog.h5')

# duration = datetime.now() - start
# print("Training completed in time: ", duration)

Epoch 1/25
625/625 [==============================] - 603s 965ms/step - loss: 1.1848 - accuracy: 0.7976 - val_loss: 1.1334 - val_accuracy: 0.7932
Epoch 2/25
625/625 [==============================] - 521s 832ms/step - loss: 1.1096 - accuracy: 0.7949 - val_loss: 1.2191 - val_accuracy: 0.7953
Epoch 3/25
625/625 [==============================] - 508s 813ms/step - loss: 1.0856 - accuracy: 0.7955 - val_loss: 1.2332 - val_accuracy: 0.7807
Epoch 4/25
625/625 [==============================] - 517s 826ms/step - loss: 1.0877 - accuracy: 0.7948 - val_loss: 0.9597 - val_accuracy: 0.8021
Epoch 5/25
625/625 [==============================] - 486s 776ms/step - loss: 1.1509 - accuracy: 0.7893 - val_loss: 1.1364 - val_accuracy: 0.7911
Epoch 6/25
625/625 [==============================] - 441s 705ms/step - loss: 1.0486 - accuracy: 0.7920 - val_loss: 1.1655 - val_accuracy: 0.7870
Epoch 7/25
625/625 [==============================] - 495s 791ms/step - loss: 1.0928 - accuracy: 0.7872 - val_loss: 1.0939 -

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [2]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from skimage import transform
import numpy as np
from keras.preprocessing import image

def load(filename):
 np_image = Image.open(filename)
 np_image = np.array(np_image).astype('float32')/255
 np_image = transform.resize(np_image, (64,64,3))
 np_image = np.expand_dims(np_image, axis=0)
 return np_image

image = 'img/IMG_6266 copy.jpg'
image_load = load(image)
result = model.predict(image_load) 
if result[0][0] > 0.5:
    prediction = 'dog'
else:
    prediction = 'cat'


img2 = mpimg.imread(image)
imgplot = plt.imshow(img2)
plt.show()

print(prediction)

Populating the interactive namespace from numpy and matplotlib


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['load']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


NameError: name 'model' is not defined